In [1]:
import sys
sys.path.append('/home/rodrigo/PycharmProjects/turbofan/')

import helper
from helper import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%matplotlib inline

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

/home/rodrigo/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/rodrigo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
datasets = []

RUL_BY_PART = True

path = '/home/rodrigo//Desktop/jaguar/Turbofan_Engine/' # path to numpy arrays to plot t-sne

df_train = pd.read_pickle(path + 'dataframes/df_train')
df_test = pd.read_pickle(path + 'dataframes/df_test')

#df_train = pd.read_pickle(path + 'dataframes/df_separated_train')
#df_test = pd.read_pickle(path + 'dataframes/df_separated_test')

if RUL_BY_PART:
    df_train['RUL'] = df_train.apply(lambda r: RUL_by_parts(r, 120), axis=1)
    df_test['RUL'] = df_test.apply(lambda r: RUL_by_parts(r, 120), axis=1)

df_train.sort_values(by=['dataset_id', 'unit_id', 'cycle'], inplace=True)
df_test.sort_values(by=['dataset_id', 'unit_id', 'cycle'], inplace=True)

df_train.reset_index(drop = True, inplace=True)
df_test.reset_index(drop = True, inplace=True)

sensor_columns = ["sensor {}".format(s+1) for s in range(21)]
columns = ['dataset_id', 'unit_id', 'cycle', 'HDBScan' , 'RUL'] + sensor_columns

df_train = df_train[columns]
df_test = df_test[columns]

df_train, df_test = cat_to_continuous_Binarizer(df_train, df_test)

In [22]:
length_sequence = 50

array_train = []
array_label = []

for x in df_train.groupby(['dataset_id', 'unit_id']).groups:
    
    df = (df_train.groupby(['dataset_id', 'unit_id']).get_group(x)).copy()
    
    for i in range(1, len(df)):
        train_x = np.zeros((length_sequence, (len(df.columns)-5)))
        train_y = np.zeros((length_sequence, 1))
        
        if i - length_sequence < 0: x = 0
        else: x = i-length_sequence
        
        data = df.iloc[x:i]
        #print(data)
        label = data['RUL'].copy().values
        data = data.drop(['dataset_id', 'unit_id', 'HDBScan', 'RUL', 'cycle'], axis=1).values
        train_x[-len(data):, :] = data
        #print(label)
        train_y[-len(data):, 0] = label
        array_train.append(train_x)
        array_label.append(train_y)

In [27]:
np.array(array_train).shape, np.array(array_label).shape

((159650, 50, 32), (159650, 50, 1))